<a href="https://www.kaggle.com/code/tiheli/ai-phone-banking-project?scriptVersionId=209608412" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telefon-bankacl-mobile-bank/bankv2.csv
/kaggle/input/telefon-bankacl-mobile-bank/banka.csv


In [2]:
# Importing necessary libraries for the text classification project
from sklearn.feature_extraction.text import CountVectorizer  # For converting text data into numerical feature vectors
from sklearn.ensemble import RandomForestClassifier  # To use a Random Forest model for text classification
from sklearn.model_selection import train_test_split  # For splitting the dataset into training and testing sets

In [3]:
df=pd.read_csv('/kaggle/input/telefon-bankacl-mobile-bank/banka.csv')

In [4]:
# Selecting only the necessary columns for the model: 'sorgu' as the text input and 'label' as the target class
df=df[['sorgu','label']]

In [5]:
# Displaying the first 3 rows of the DataFrame to review the structure and content of 'sorgu' and 'label' columns
df.head(3)

,sorgu,label
0,kredi kartımı kaybettim,kartkayip
1,banka kartımı kaybettim,kartkayip
2,kredi kartımı bulamıyorum,kartkayip


https://www.cagrigungor.com/turkish-stopwords-turkce-rnn-lstm-stopwords-kelime-listesi/

# Filtering the DataFrame to keep only 'sorgu' (query) and 'label' columns
paradf = df[['sorgu', 'label']]

# Defining a list of Turkish stopwords to remove common, non-informative words from text data
stopwords = ['fakat', 'lakin', 'ancak', 'acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 
             'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 
             'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 
             'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 
             'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']

# Getting a new text input from the user to classify
massage = input("Enter the action you want to perform: ")

# Creating a new DataFrame for the input message with a dummy label for preprocessing
msgdf = pd.DataFrame({"sorgu": massage, "label": 0}, index=[42])

# Concatenating the new message to the main DataFrame for processing
df = pd.concat([df, msgdf], ignore_index=True)

# Removing stopwords from the 'sorgu' text data by replacing them with spaces
for word in stopwords:
    word = " " + word + " "
    df['sorgu'] = df['sorgu'].str.replace(word, " ")

# Initializing CountVectorizer to convert text data into a numerical feature matrix with the top 50 words
cv = CountVectorizer(max_features=50)
x = cv.fit_transform(df['sorgu']).toarray()  # Transforming 'sorgu' column into a matrix of token counts
y = df['label']  # Defining the labels as target variable 'y'

# Extracting the feature matrix for the user-input message for prediction
pred = x[-1].copy()  # Saving the last row (user input) separately for future prediction

# Removing the user-input message from the training data
x = x[0:-1]
y = y[0:-1]

# Splitting the data into training and testing sets with 70% for training
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=21, train_size=0.7)

# Initializing and training a Random Forest Classifier on the training data
rf = RandomForestClassifier()
model = rf.fit(x_train, y_train)  # Fitting the model

# Scoring the model on the test data to evaluate accuracy
score = model.score(x_test, y_test)

# Predicting the label for the user-input message using the trained model
result = model.predict([pred])

# Displaying the predicted label for the input message along with the model's accuracy score
print("Result:", result, "Score:", score)